In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import os
import os.path
import statistics as stat

### first rename file, only run this once

In [ ]:
datapath = '/Users/sijinchen/Desktop/new data/new summary data_RL/'

filenameList = []
IDlist = []
subjectID = []
session_list = []

number = 0
for file in os.listdir(datapath):
    filename = os.path.join(datapath, file)

    ##check if the file is csv file
    y = (filename.split('/'))[-1][-3:]
    if y == 'csv':
        number += 1
        IDlist.append(number)
        
        x = (filename.split('/'))[-1][:-4]
        filenameList.append(x)
        ID = x.split('_')[0]
        subjectID.append(ID)
        session = x.split('_')[2]
        session_list.append(session)
        os.rename (filename, datapath+'/'+ str(number)+ '.csv')

Dict = { 'numeric ID': IDlist,'filename': filenameList, 'ID': subjectID, 'session_type': session_list}
df = pd.DataFrame(Dict)
df.to_csv('real ID to numeric ID 2.csv')

### these are functions for 5 models: random with bias, win stay lose shift, RL, RL + choice kernel, asymmetrical RL + choice kernel. For now, you can look at the RL and RL+choice kernel model parameters.

In [ ]:
def random_model(theta):  
    
    datapath = '/Users/sijinchen/Desktop/Cosyne 2024 data analysis/summary data for RL/'
    
    df = pd.read_csv(datapath + '//' + str(number) +'.csv')

    sideList= df['choice'].tolist()
    rewardList = df['outcome'].tolist()

    bias1 = theta[0]

    probList=[]

    for i in range (len(rewardList)):
        arm = int(sideList[i])

        if arm == 0:
            prob1 = bias1
            probList.append(prob1)

        if arm == 1:
            prob2 = (1-bias1)/2
            probList.append(prob2)
        
        if arm == 2:
            prob3 = (1-bias1)/2
            probList.append(prob3)


        ##log likelihood
    logLike = (np.sum(np.log(probList)))*(-1)
    return logLike

In [ ]:
def wsls_model(theta):  
    
    datapath = '/Users/sijinchen/Desktop/Cosyne 2024 data analysis/summary data for RL/'
    
    df = pd.read_csv(datapath + '//' + str(number) +'.csv')

    sideList= df['choice'].tolist()
    rewardList = df['outcome'].tolist()

    epsilon = theta[0]

    probList=[]

    for i in range (1, len(rewardList)):
        if int(sideList[i]) ==  int(sideList[i-1]) and rewardList[i-1] == 1:
            prob = 1 - epsilon/2
            probList.append(prob)
        elif int(sideList[i]) !=  int(sideList[i-1]) and rewardList[i-1] == 0:
            prob = 1 - epsilon/2
            probList.append(prob)
        elif int(sideList[i]) !=  int(sideList[i-1]) and rewardList[i-1] == 1:
            prob = epsilon/2
            probList.append(prob)
        elif int(sideList[i]) ==  int(sideList[i-1]) and rewardList[i-1] == 0:
            prob = epsilon/2
            probList.append(prob)

        ##log likelihood
    logLike = (np.sum(np.log(probList)))*(-1)

    return logLike

In [2]:
def rl_model(theta):  
    
    datapath = '/Users/sijinchen/Desktop/Cathy Caroline paper/summary data for RL/'
    
    df = pd.read_csv(datapath + '//' + str(number) +'.csv')

    sideList= df['choice'].tolist()
    rewardList = df['outcome'].tolist()

    Qvalue = [0,0,0]

    alpha = theta[0]
    beta = theta[1]

    probList=[]

    for i in range (len(rewardList)):
        arm = int(sideList[i])

        Qvalue [arm] += alpha*(rewardList[i]-Qvalue[arm])

            ##Softmax function + choice kernel here
        if arm == 0:
            prob1 = (np.exp(Qvalue[0]*beta))/(np.exp((Qvalue[0]*beta)) + (np.exp(Qvalue[1]*beta)) + (np.exp(Qvalue[2]*beta)))
            probList.append(prob1)

        if arm == 1:
            prob2 = (np.exp(Qvalue[1]*beta))/(np.exp((Qvalue[0]*beta)) + (np.exp(Qvalue[1]*beta)) + (np.exp(Qvalue[2]*beta)))
            probList.append(prob2)

        if arm == 2:
            prob3 = (np.exp(Qvalue[2]*beta))/(np.exp((Qvalue[0]*beta)) + (np.exp(Qvalue[1]*beta)) + (np.exp(Qvalue[2]*beta)))
            probList.append(prob3)


        ##log likelihood
    logLike = (np.sum(np.log(probList)))*(-1)

    return logLike

In [ ]:
def rlck_model(theta):  
    
    datapath = '/Users/sijinchen/Desktop/Cathy Caroline paper/summary data for RL/'
    
    df = pd.read_csv(datapath + '//' + str(number) +'.csv')

    sideList= df['choice'].tolist()
    rewardList = df['outcome'].tolist()

    Qvalue = [0,0,0]
    CKvalue = [0,0,0]

    alpha = theta[0]
    beta = theta[1]

    alphaC = theta [2]
    tau = theta [3]

    probList=[]

    for i in range (len(rewardList)):
        arm = int(sideList[i])

        Qvalue [arm] += alpha*(rewardList[i]-Qvalue[arm])
        CKvalue [arm] += alphaC*(1-CKvalue[arm])

            ##Softmax function + choice kernel here
        if arm == 0:
            prob1 = (np.exp((Qvalue[0]*tau + (1-tau)*CKvalue[0])*beta))/((np.exp((Qvalue[0]*tau + (1-tau)*CKvalue[0])*beta)) + 
                (np.exp((Qvalue[1]*tau + (1-tau)*CKvalue[1])*beta))+ (np.exp((Qvalue[2]*tau + (1-tau)*CKvalue[2])*beta)))
            probList.append(prob1)

        if arm == 1:
            prob2 = (np.exp((Qvalue[1]*tau + (1-tau)*CKvalue[1])*beta))/((np.exp((Qvalue[0]*tau + (1-tau)*CKvalue[0])*beta)) + 
                (np.exp((Qvalue[1]*tau + (1-tau)*CKvalue[1])*beta))+ (np.exp((Qvalue[2]*tau + (1-tau)*CKvalue[2])*beta)))
            probList.append(prob2)

        if arm == 2:
            prob3 = (np.exp((Qvalue[2]*tau + (1-tau)*CKvalue[2])*beta))/((np.exp((Qvalue[0]*tau + (1-tau)*CKvalue[0])*beta)) + 
                (np.exp((Qvalue[1]*tau + (1-tau)*CKvalue[1])*beta))+ (np.exp((Qvalue[2]*tau + (1-tau)*CKvalue[2])*beta)))
            probList.append(prob3)
        
        ##log likelihood
    logLike = (np.sum(np.log(probList)))*(-1)
        
    return logLike

In [ ]:
def rlck_model2(theta):  
    
    datapath = '/Users/sijinchen/Desktop/new data/new summary data_RL/'
    
    df = pd.read_csv(datapath + '//' + str(number) +'.csv')

    sideList= df['choice'].tolist()
    rewardList = df['outcome'].tolist()

    Qvalue = [0,0,0]
    CKvalue = [0,0,0]

    alpha = theta[0]
    beta = theta[1]

    alphaC = theta [2]
    tau = theta [3]

    probList=[0.3]

    for i in range (1, len(rewardList)):
        arm_previous = int(sideList[i-1])
        arm_current = int(sideList[i])

        Qvalue [arm_previous] += alpha*(rewardList[i-1]-Qvalue[arm_previous])
        CKvalue [arm_previous] += alphaC*(1-CKvalue[arm_previous])

            ##Softmax function + choice kernel here
        if arm_current == 0:
            prob1 = (np.exp((Qvalue[0]*tau + (1-tau)*CKvalue[0])*beta))/((np.exp((Qvalue[0]*tau + (1-tau)*CKvalue[0])*beta)) + 
                (np.exp((Qvalue[1]*tau + (1-tau)*CKvalue[1])*beta))+ (np.exp((Qvalue[2]*tau + (1-tau)*CKvalue[2])*beta)))
            probList.append(prob1)

        if arm_current == 1:
            prob2 = (np.exp((Qvalue[1]*tau + (1-tau)*CKvalue[1])*beta))/((np.exp((Qvalue[0]*tau + (1-tau)*CKvalue[0])*beta)) + 
                (np.exp((Qvalue[1]*tau + (1-tau)*CKvalue[1])*beta))+ (np.exp((Qvalue[2]*tau + (1-tau)*CKvalue[2])*beta)))
            probList.append(prob2)

        if arm_current == 2:
            prob3 = (np.exp((Qvalue[2]*tau + (1-tau)*CKvalue[2])*beta))/((np.exp((Qvalue[0]*tau + (1-tau)*CKvalue[0])*beta)) + 
                (np.exp((Qvalue[1]*tau + (1-tau)*CKvalue[1])*beta))+ (np.exp((Qvalue[2]*tau + (1-tau)*CKvalue[2])*beta)))
            probList.append(prob3)
        
        ##log likelihood
    logLike = (np.sum(np.log(probList)))*(-1)
        
    return logLike

In [ ]:
def rlck_2states_model(theta):  
    
    datapath = '/Users/sijinchen/Desktop/Cathy Caroline paper/summary data for RL/'
    statePath = '/Users/sijinchen/Desktop/Cathy Caroline paper/state label/'
    
    df = pd.read_csv(datapath + '//' + str(number) +'.csv')

    ID = df['user id'].tolist()[0]
    session = df['session_type'].tolist()[0]
    sideList= df['choice'].tolist()
    rewardList = df['outcome'].tolist()

    state = pd.read_csv ( statePath + ID + '_TBT_' + session + '_stateLabel.csv')['state'].tolist()

    Qvalue = [0,0,0]
    CKvalue = [0,0,0]

    alpha_ore = theta[0]
    alpha_oit = theta[1]
    beta = theta[2]

    alphaC = theta [3]
    betaC = theta[4]

    probList=[0.3]

    for i in range (1, len(rewardList)):
        arm_previous = int(sideList[i-1])
        arm_current = int(sideList[i])

        if state[i-1] == 0: ##explore
            Qvalue [arm_previous] += alpha_ore*(rewardList[i-1]-Qvalue[arm_previous])
            CKvalue [arm_previous] += alphaC*(1-CKvalue[arm_previous])
        else:
            Qvalue [arm_previous] += alpha_oit*(rewardList[i-1]-Qvalue[arm_previous])
            CKvalue [arm_previous] += alphaC*(1-CKvalue[arm_previous])

            ##Softmax function + choice kernel here
        if arm_current == 0:
            prob1 = (np.exp(Qvalue[0]*beta + CKvalue[0]*betaC))/(np.exp(Qvalue[0]*beta + CKvalue[0]*betaC) + 
                np.exp(Qvalue[1]*beta + CKvalue[1]*betaC)+ np.exp(Qvalue[2]*beta + CKvalue[2]*betaC))
            probList.append(prob1)

        if arm_current == 1:
            prob2 = (np.exp(Qvalue[1]*beta + CKvalue[1]*betaC))/(np.exp(Qvalue[0]*beta + CKvalue[0]*betaC) + 
                np.exp(Qvalue[1]*beta + CKvalue[1]*betaC)+ np.exp(Qvalue[2]*beta + CKvalue[2]*betaC))
            probList.append(prob2)

        if arm_current == 2:
            prob3 = (np.exp(Qvalue[2]*beta + CKvalue[2]*betaC))/(np.exp(Qvalue[0]*beta + CKvalue[0]*betaC) + 
                np.exp(Qvalue[1]*beta + CKvalue[1]*betaC)+ np.exp(Qvalue[2]*beta + CKvalue[2]*betaC))
            probList.append(prob3)
        
        ##log likelihood
    logLike = (np.sum(np.log(probList)))*(-1)
        
    return logLike

In [ ]:
def rlckg_model(theta):  
    
    datapath = '/Users/sijinchen/Desktop/new data/new summary data_RL/'
    
    df = pd.read_csv(datapath + '//' + str(number) +'.csv')

    sideList= df['choice'].tolist()
    rewardList = df['outcome'].tolist()
        
    Qvalue = [0,0,0]
    CKvalue = [0,0,0]
    Fvalue = [0,0,0]

    alpha = theta[0]
    beta = theta[1]
    alphaC = theta [2]
    gamma = theta [3]

    probList=[]

    for i in range (len(rewardList)):
        arm = int(sideList[i])

        if rewardList[i] == 1:
            Qvalue [arm] += alpha*(rewardList[i]-Qvalue[arm])

        elif rewardList[i] == 0:
            Qvalue [arm] += gamma*alpha*(rewardList[i]-Qvalue[arm])


        CKvalue [arm] += alphaC*(1-CKvalue[arm])
        Fvalue[arm] = Qvalue[arm] + CKvalue[arm]

            ##Softmax function + choice kernel here
        if arm == 0:
            prob1 = (np.exp((Fvalue[0])*beta))/((np.exp((Fvalue[0])*beta))+(np.exp((Fvalue[1])*beta))+(np.exp((Fvalue[2])*beta)))
            probList.append(prob1)
            #print (prob1)

        elif arm == 1:
            prob2 = (np.exp((Fvalue[1])*beta))/((np.exp((Fvalue[0])*beta))+(np.exp((Fvalue[1])*beta))+(np.exp((Fvalue[2])*beta)))
            probList.append(prob2)

        elif arm == 2:
            prob3 = (np.exp((Fvalue[2])*beta))/((np.exp((Fvalue[0])*beta))+(np.exp((Fvalue[1])*beta))+(np.exp((Fvalue[2])*beta)))
            probList.append(prob3)

        ##log likelihood
    logLike = (np.sum(np.log(probList)))*(-1)
        
    return logLike

In [ ]:
def forage_model(theta):  
    
    datapath = '/Users/sijinchen/Desktop/new data/new summary data_RL/'
    
    df = pd.read_csv(datapath + '//' + str(number) +'.csv')

    sideList= df['choice'].tolist() 
    rewardList = df['outcome'].tolist()
    oitOreList= []
    
    #switch/explore = 0, stay/exploit = 1
    for j in range (len(sideList)-1):
        if sideList[j]==sideList[j+1]:
            oitOreList.append(1)
        else:
            oitOreList.append(0)

    rewardList = rewardList[1:]
            
    v_oit = 0

    alpha = theta[0]
    beta = theta[1]   
    rho = theta[2]
    gamma = theta[3]

    probList=[]
    for i in range (len(rewardList)):
        arm = oitOreList[i]

        if arm == 1:
            v_oit += alpha*(rewardList[i]-v_oit) + gamma*(1-v_oit)

        if arm == 1:
            prob = 1/(1+(np.exp(-1*(v_oit-rho)*beta)))
            probList.append(prob)

        if arm == 0:
            prob = 1-1/(1+(np.exp(-1*(v_oit-rho)*beta)))
            probList.append(prob)
            v_oit = 0.5

        ##log likelihood
    logLike = (np.sum(np.log(probList)))*(-1)
    return logLike


### here are the functions for optimization step of the 5 models (step of model-fitting to real data). Don't forgot to change the number to the number of subject/file to analyze

In [ ]:
def main_random():
    nll = []
    bias1List = []
    global number
    for number in range (1,337): ### change to the number of participant +1 

        replication = 30
        bnds = ((0,1), )

        for h in range (replication):
            # print ("*************************************************")
            # print ("This is replication number " + str(h))
            bias1 = np.random.random()

            startParams = [bias1]
            theta_optim = minimize(random_model, startParams, method = "TNC", bounds = bnds, tol = 0.001)

            if h == 0:
                minimizeLog = theta_optim.fun
                optimX = theta_optim.x
            else:
                if theta_optim.fun < minimizeLog:
                    print ('**********')
                    minimizeLog = theta_optim.fun
                    optimX = theta_optim.x

        nll.append(minimizeLog)
        theta =  (optimX.tolist())
        bias1List.append(theta[0])

        dataDict = { 'bias1': bias1List,  'model likelihood': nll}
        data_df = pd.DataFrame(dataDict)
        data_df.to_csv('model1_random optimized parameters.csv')

In [ ]:
def main_wsls():
    nll = []
    epsilonList = []

    global number
    for number in range (1,337):

        replication = 30
        bnds = ((0,1), )

        for h in range (replication):
            print ("*************************************************")
            print ("This is replication number " + str(h))
            epsilon = np.random.random()

            startParams = [epsilon]
            theta_optim = minimize(wsls_model, startParams, method = "TNC", bounds = bnds, tol = 0.001)

            if h == 0:
                minimizeLog = theta_optim.fun
                optimX = theta_optim.x
            else:
                if theta_optim.fun < minimizeLog:
                    print ('**********')
                    minimizeLog = theta_optim.fun
                    optimX = theta_optim.x

        nll.append(minimizeLog)
        theta =  (optimX.tolist())
        epsilonList.append(theta[0])

        dataDict = { 'epsilon': epsilonList, 'model likelihood': nll}
        data_df = pd.DataFrame(dataDict)
        data_df.to_csv('model2_wsls optimized parameters.csv')

In [3]:
def main_RL():
    nll = []
    alphaList = []
    betaList = []

    global number
    for number in range (1,410):

        replication = 30
        bnds = ((0,1),(0, float('inf')), )

        for h in range (replication):
            print ("*************************************************")
            print ("This is replication number " + str(h))
            alpha = np.random.random()
            beta = np.random.exponential(scale =1, size = None)

            startParams = [alpha, beta]
            theta_optim = minimize(rl_model, startParams, method = "TNC", bounds = bnds, tol = 0.00001)

            if h == 0:
                minimizeLog = theta_optim.fun
                optimX = theta_optim.x
            else:
                if theta_optim.fun < minimizeLog:
                    print ('**********')
                    minimizeLog = theta_optim.fun
                    optimX = theta_optim.x

        nll.append(minimizeLog)
        theta =  (optimX.tolist())
        alphaList.append(theta[0])
        betaList.append(theta[1])

        dataDict = { 'alpha': alphaList, 'beta': betaList,  'model likelihood': nll}
        data_df = pd.DataFrame(dataDict)
        data_df.to_csv('model3_RL optimized parameters_TBT.csv')

In [ ]:
def main_RLCK():
    nll = []
    alphaList = []
    betaList = []
    alphaCList = []
    tauList = []

    global number
    for number in range (1,337):

        replication = 30
        bnds = ((0,1),(0, float('inf')), (-1,1), (0,1))

        for h in range (replication):
            print ("*************************************************")
            print ("This is replication number " + str(h))
            alpha = np.random.random()
            beta = np.random.exponential(scale =1, size = None)

            alphaC = np.random.random()
            tau = np.random.random()

            startParams = [alpha, beta, alphaC, tau]
            theta_optim = minimize(rlck_model, startParams, method = "TNC", bounds = bnds, tol = 0.00001)

            if h == 0:
                minimizeLog = theta_optim.fun
                optimX = theta_optim.x
            else:
                print (theta_optim.x)
                print (theta_optim.fun)
                if theta_optim.fun < minimizeLog:
                    print ('**********')
                    minimizeLog = theta_optim.fun
                    optimX = theta_optim.x

        nll.append(minimizeLog)
        theta =  (optimX.tolist())
        alphaList.append(theta[0])
        betaList.append(theta[1])
        alphaCList.append(theta[2])
        tauList.append(theta[3])

        # dataDict = { 'alpha': alphaList, 'beta': betaList, 'alphaC': alphaCList, 'choice value tuner': tauList, 'model likelihood': nll}
        # data_df = pd.DataFrame(dataDict)
        # data_df.to_csv('model4_RLCK optimized parameters.csv')


In [ ]:
def main_RLCK2():
    nll = []
    alphaList = []
    betaList = []
    alphaCList = []
    tauList = []

    global number
    for number in range (1,74):

        replication = 30
        bnds = ((0,1),(0, float('inf')), (0,1), (0,1))

        for h in range (replication):
            print ("*************************************************")
            print ("This is replication number " + str(h))
            alpha = np.random.random()
            beta = np.random.exponential(scale =1, size = None)

            alphaC = np.random.random()
            tau = np.random.random()

            startParams = [alpha, beta, alphaC, tau]
            theta_optim = minimize(rlck_model2, startParams, method = "TNC", bounds = bnds, tol = 0.00001)

            if h == 0:
                minimizeLog = theta_optim.fun
                optimX = theta_optim.x
            else:
                print (theta_optim.x)
                print (theta_optim.fun)
                if theta_optim.fun < minimizeLog:
                    print ('**********')
                    minimizeLog = theta_optim.fun
                    optimX = theta_optim.x

        nll.append(minimizeLog)
        theta =  (optimX.tolist())
        alphaList.append(theta[0])
        betaList.append(theta[1])
        alphaCList.append(theta[2])
        tauList.append(theta[3])

        dataDict = { 'alpha': alphaList, 'beta': betaList, 'alphaC': alphaCList, 'choice value tuner': tauList, 'model likelihood': nll}
        data_df = pd.DataFrame(dataDict)
        data_df.to_csv('model4_RLCK optimized parameters_2.csv')


In [ ]:
def main_rlck_2states():
    nll = []
    alpha_oreList = []
    alpha_oitList = []
    betaList = []
    alphaCList = []
    betaCList = []

    global number
    for number in range (1,410):

        replication = 20
        bnds = ((0,1),(0,1),(0, float('inf')), (0,1), (0, float('inf')),)

        for h in range (replication):
            print ("*************************************************")
            print ("This is replication number " + str(h))
            alpha_ore = np.random.random()
            alpha_oit = np.random.random()
            beta = np.random.exponential(scale =1, size = None)
            alphaC = np.random.random()
            betaC = np.random.random()

            startParams = [alpha_ore, alpha_oit, beta, alphaC, betaC]
            theta_optim = minimize(rlck_2states_model, startParams, method = "TNC", bounds = bnds, tol = 0.0001)

            if h == 0:
                minimizeLog = theta_optim.fun
                optimX = theta_optim.x
            else:
                print (theta_optim.x)
                print (theta_optim.fun)
                if theta_optim.fun < minimizeLog:
                    print ('**********')
                    minimizeLog = theta_optim.fun
                    optimX = theta_optim.x

        nll.append(minimizeLog)
        theta =  (optimX.tolist())
        alpha_oreList.append(theta[0])
        alpha_oitList.append(theta[1])
        betaList.append(theta[2])
        alphaCList.append(theta[3])
        betaCList.append(theta[4])

        dataDict = { 'alpha_ore': alpha_oreList,'alpha_oit': alpha_oitList, 'beta': betaList, 'alphaC': alphaCList, 'betaC': betaCList, 'model likelihood': nll}
        data_df = pd.DataFrame(dataDict)
        data_df.to_csv('model8_RLCK 2 states optimized parameters.csv')


In [ ]:
def main_RLCKG():
    nll = []
    alphaList = []
    betaList = []
    alphaCList = []
    gammaList = []

    global number
    for number in range (1,74):

        replication = 30
        bnds = ((0,1),(0, float('inf')), (0,1), (0,1))

        for h in range (replication):
            print ("*************************************************")
            print ("This is replication number " + str(h))
            alpha = np.random.random()
            beta = np.random.exponential(scale =1, size = None)
            alphaC = np.random.random()
            gamma = np.random.random()

            startParams = [alpha, beta, alphaC, gamma]
            theta_optim = minimize(rlckg_model, startParams, method = "TNC", bounds = bnds, tol = 0.00001)

            if h == 0:
                minimizeLog = theta_optim.fun
                optimX = theta_optim.x
            else:
                if theta_optim.fun < minimizeLog:
                    print ('**********')
                    minimizeLog = theta_optim.fun
                    optimX = theta_optim.x

        nll.append(minimizeLog)
        theta =  (optimX.tolist())
        alphaList.append(theta[0])
        betaList.append(theta[1])
        alphaCList.append(theta[2])
        gammaList.append(theta[3])

        dataDict = { 'alpha': alphaList, 'beta': betaList, 'alphaC': alphaCList, 'asymetric learning rate': gammaList, 'model likelihood': nll}
        data_df = pd.DataFrame(dataDict)
        data_df.to_csv('model5_RLCKG optimized parameters.csv')


In [ ]:
def main_foraging():
    nll = []
    alphaList = []
    betaList = []
    rhoList = []
    gammaList = []

    global number
    for number in range (1,74):

        replication = 50
        bnds = ((0,1),(0, float('inf')), (0,1), (-1,1),)

        for h in range (replication):
            print ("*************************************************")
            print ("This is replication number " + str(h))
            alpha = np.random.random()
            beta = np.random.exponential(scale =1, size = None)
            rho = np.random.random()
            gamma = np.random.uniform(-1,1,1)

            startParams = [alpha, beta,rho,gamma]
            theta_optim = minimize(forage_model, startParams, method = "TNC", bounds = bnds, tol = 0.00001)

            if h == 0:
                minimizeLog = theta_optim.fun
                optimX = theta_optim.x
            else:
                if theta_optim.fun < minimizeLog:
                    print ('**********')
                    minimizeLog = theta_optim.fun
                    optimX = theta_optim.x

        nll.append(minimizeLog)
        theta =  (optimX.tolist())
        alphaList.append(theta[0])
        betaList.append(theta[1])
        rhoList.append(theta[2])
        gammaList.append(theta[3])

        dataDict = { 'alpha': alphaList, 'beta': betaList, 'threshold': rhoList, 'bias': gammaList, 'model likelihood': nll}
        data_df = pd.DataFrame(dataDict)
        data_df.to_csv('model6_foraging optimized parameters.csv')
    

### run each model here

In [ ]:
main_random()

In [ ]:
main_wsls()

In [4]:
main_RL()

*************************************************
This is replication number 0
*************************************************
This is replication number 1
**********
*************************************************
This is replication number 2
**********
*************************************************
This is replication number 3
**********
*************************************************
This is replication number 4
*************************************************
This is replication number 5
*************************************************
This is replication number 6
*************************************************
This is replication number 7
*************************************************
This is replication number 8
*************************************************
This is replication number 9
*************************************************
This is replication number 10
**********
*************************************************
This is replication number 11
******

/var/folders/4c/qtyzwxns1jz7zlm880prvytw0000gn/T/ipykernel_43003/601065326.py:24: RuntimeWarning: overflow encountered in exp
  prob1 = (np.exp(Qvalue[0]*beta))/(np.exp((Qvalue[0]*beta)) + (np.exp(Qvalue[1]*beta)) + (np.exp(Qvalue[2]*beta)))
/var/folders/4c/qtyzwxns1jz7zlm880prvytw0000gn/T/ipykernel_43003/601065326.py:24: RuntimeWarning: invalid value encountered in double_scalars
  prob1 = (np.exp(Qvalue[0]*beta))/(np.exp((Qvalue[0]*beta)) + (np.exp(Qvalue[1]*beta)) + (np.exp(Qvalue[2]*beta)))
/var/folders/4c/qtyzwxns1jz7zlm880prvytw0000gn/T/ipykernel_43003/601065326.py:28: RuntimeWarning: overflow encountered in exp
  prob2 = (np.exp(Qvalue[1]*beta))/(np.exp((Qvalue[0]*beta)) + (np.exp(Qvalue[1]*beta)) + (np.exp(Qvalue[2]*beta)))
/var/folders/4c/qtyzwxns1jz7zlm880prvytw0000gn/T/ipykernel_43003/601065326.py:28: RuntimeWarning: invalid value encountered in double_scalars
  prob2 = (np.exp(Qvalue[1]*beta))/(np.exp((Qvalue[0]*beta)) + (np.exp(Qvalue[1]*beta)) + (np.exp(Qvalue[2]*beta)))


*************************************************
This is replication number 8
*************************************************
This is replication number 9
**********
*************************************************
This is replication number 10
*************************************************
This is replication number 11
*************************************************
This is replication number 12
*************************************************
This is replication number 13
*************************************************
This is replication number 14
*************************************************
This is replication number 15
*************************************************
This is replication number 16
*************************************************
This is replication number 17
*************************************************
This is replication number 18
*************************************************
This is replication number 19
**********
********************

In [ ]:
main_RLCK2()

In [ ]:
main_RLCK()

In [ ]:
main_RLCKG()

In [ ]:
main_foraging()

In [ ]:
main_rlck_2states()